## Лабораторная работа №2. Предварительная обработка текстовых данных

**Выполнил:** Китайский А.С.

**Проверил:** Мохов А.С.

### 1. Импорт необходимых модулей

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import numpy as np
import pandas as pd

### 2. Загрузка данных

Датасет «20 news groups» состоит приблизительно из 18000 сообщений на английском языке по 20 тематикам

| Вариант | Класс     |
|---------|-----------|
| 8       | 5, 16, 20 |

| № класса | Название класса          |
|----------|--------------------------|
| 5        | 'comp.sys.mac.hardware'  |
| 16       | 'soc.religion.christian' |
| 20       | 'talk.religion.misc'     |

In [11]:
categories = ['comp.sys.mac.hardware', 'soc.religion.christian', 'talk.religion.misc'] 
remove = ('headers', 'footers', 'quotes')
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories, remove=remove)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, categories=categories, remove=remove)

### 3. Выведем по одному документа каждого класса

In [13]:
for i in range(len(categories)):
    print(f'Документ класса {categories[i]}:')
    print(twenty_train.data[np.where(twenty_train.target == i)[0][0]], 
          end='\n-------------------------------------------------\n')

Документ класса comp.sys.mac.hardware:
A SIMM is a small PCB with DRAM chips soldered on.

--maarten
-------------------------------------------------
Документ класса soc.religion.christian:

[deletia- and so on]

I seem to have been rather unclear.

What I was asking is this:

Please show me that the most effective substance-absure recovery
programs involve meetinsg peoples' spiritual needs, rather than
merely attempting to fill peoples' spiritual needs as percieved
by the people, A.A, S.R.C. regulars, or snoopy. This will probably
involve defining "spritual needs" (is it not that clear) and
showing that such things exist and how they can be filled.

Annother tack you might take is to say that "fulfilling spiritual
needs" means "acknowledging a "higher power" of some sort, then
show that systems that do require this, work better than otherwise
identical systems that do not. A correlation here would help you,
but as you point out this might just be demonstrating swapping
one crutch for

In [14]:
for i in range(len(categories)):
    print(f'Количество объектов класса {categories[i]}:', end=' ')
    print(np.unique(twenty_train.target, return_counts=True)[1][i])

Количество объектов класса comp.sys.mac.hardware: 578
Количество объектов класса soc.religion.christian: 599
Количество объектов класса talk.religion.misc: 377


Классы более менее сбалансированы

### 4. Применим стемминг и запишем обработанные выборки в новые переменные

In [15]:
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from tqdm import tqdm

porter_stemmer = PorterStemmer()
def stem_text(text):
    nltk_tokens = word_tokenize(text)
    line = ' '.join([porter_stemmer.stem(word) for word in nltk_tokens])
    return line

stem_train = list(tqdm(map(stem_text, twenty_train.data), total=len(twenty_train.data)))
stem_test = list(tqdm(map(stem_text, twenty_test.data), total=len(twenty_test.data)))

100%|██████████| 1034/1034 [00:04<00:00, 229.60it/s]


### 5. Проведем векторизацию выборки: a. Векторизуем обучающую и тестовую выборки простым подсчетом слов (CountVectorizer) b. Выведем первые 20 наиболее частотных слов всей выборки и каждого класса по-отдельности. c. Рассчитаем сходство по коэффициенту Жаккара между тремя классами d. Применим процедуру отсечения стоп-слов и повторим пункты b-c. e. Проведем пункты a – c для обучающей и тестовой выборки, для которой проведена процедура стемминга.

In [67]:
from nltk.corpus import stopwords
mystopwords = stopwords.words('english') 

In [77]:
def print_frequency_words(train, class_num=None, stop_words=None):
    '''Векторизация выборки и вывод наиболее частотных слов'''
    
    if class_num is not None:
        train = [twenty_train.data[i] for i in np.where(twenty_train.target == class_num)[0]]

    vect = CountVectorizer(max_features=10000, stop_words=stop_words)
    train_data = vect.fit_transform(train)

    x = list(zip(vect.get_feature_names_out(), np.ravel(train_data.sum(axis=0))))
    x.sort(key=lambda row: row[1], reverse=True) 
    if class_num is not None:
        print(f'Первые 20 наиболее частотных слов класса {twenty_train.target_names[class_num]}:')
    else:
        print(f'Первые 20 наиболее частотных слов всей выборки:')
    print(x[:20])
    

<img src="./jacc_coef.png" width="500">

In [78]:
def calc_jaccard_coef(first_list, second_list):
    '''Рассчет коэффициента Жаккара'''
    first_set = set(first_list)
    second_set = set(second_list)
    total = len(first_set.intersection(second_set))
    coef = total / (len(first_set) + len(second_set) - total)
    return coef

def print_jaccard_coef(data, c0, c1, stop_words=None):
    class0 = [data[i] for i in np.where(twenty_train.target == c0)[0]]
    class1 = [data[i] for i in np.where(twenty_train.target == c1)[0]]
    vect0 = CountVectorizer(max_features=10000, stop_words=stop_words).fit(class0)
    vect1 = CountVectorizer(max_features=10000, stop_words=stop_words).fit(class1)
    print(f'Коэффициент Жаккара между {c0} и {c1} классами:', end='\t')
    print(round(calc_jaccard_coef(vect0.get_feature_names_out(), vect1.get_feature_names_out()), 3))

#### Без стемминга и без отсечения стоп слов

In [79]:
print_frequency_words(twenty_train.data, class_num=None)
print_frequency_words(twenty_train.data, class_num=0)
print_frequency_words(twenty_train.data, class_num=1)
print_frequency_words(twenty_train.data, class_num=2)

print_jaccard_coef(twenty_train.data, 0, 1)
print_jaccard_coef(twenty_train.data, 0, 2)
print_jaccard_coef(twenty_train.data, 1, 2)

Первые 20 наиболее частотных слов всей выборки:
[('the', 16652), ('to', 8490), ('of', 8334), ('and', 6656), ('that', 5747), ('is', 5591), ('in', 4801), ('it', 3830), ('you', 3092), ('not', 2749), ('for', 2699), ('this', 2486), ('be', 2316), ('are', 2220), ('have', 2166), ('as', 2136), ('with', 2071), ('on', 1823), ('but', 1818), ('was', 1622)]
Первые 20 наиболее частотных слов класса comp.sys.mac.hardware:
[('the', 3290), ('to', 1544), ('and', 1248), ('of', 972), ('is', 966), ('it', 873), ('in', 748), ('for', 731), ('that', 706), ('with', 622), ('have', 534), ('on', 532), ('you', 531), ('this', 482), ('be', 410), ('if', 402), ('but', 361), ('or', 358), ('can', 357), ('not', 347)]
Первые 20 наиболее частотных слов класса soc.religion.christian:
[('the', 8723), ('of', 4787), ('to', 4584), ('and', 3422), ('that', 3331), ('is', 3177), ('in', 2707), ('it', 1994), ('not', 1617), ('you', 1499), ('this', 1364), ('be', 1333), ('for', 1317), ('as', 1254), ('are', 1241), ('god', 1097), ('have', 1

Наиболее частотными словами для всех классов являются стоп-слова

#### Без стемминга и с отсечением стоп слов

In [80]:
print_frequency_words(twenty_train.data, class_num=None, stop_words=mystopwords)
print_frequency_words(twenty_train.data, class_num=0, stop_words=mystopwords)
print_frequency_words(twenty_train.data, class_num=1, stop_words=mystopwords)
print_frequency_words(twenty_train.data, class_num=2, stop_words=mystopwords)

print_jaccard_coef(twenty_train.data, 0, 1, stop_words=mystopwords)
print_jaccard_coef(twenty_train.data, 0, 2, stop_words=mystopwords)
print_jaccard_coef(twenty_train.data, 1, 2, stop_words=mystopwords)

Первые 20 наиболее частотных слов всей выборки:
[('god', 1427), ('one', 1140), ('would', 1057), ('people', 779), ('jesus', 722), ('know', 625), ('think', 565), ('like', 559), ('also', 515), ('say', 461), ('may', 456), ('us', 453), ('time', 444), ('believe', 437), ('good', 416), ('church', 414), ('bible', 411), ('even', 410), ('see', 399), ('christian', 396)]
Первые 20 наиболее частотных слов класса comp.sys.mac.hardware:
[('mac', 327), ('apple', 266), ('one', 246), ('drive', 211), ('would', 200), ('get', 178), ('use', 173), ('problem', 171), ('like', 163), ('know', 162), ('bit', 150), ('scsi', 142), ('system', 139), ('anyone', 123), ('thanks', 120), ('card', 115), ('32', 113), ('memory', 112), ('new', 110), ('also', 109)]
Первые 20 наиболее частотных слов класса soc.religion.christian:
[('god', 1097), ('would', 610), ('one', 608), ('jesus', 466), ('people', 452), ('church', 340), ('think', 337), ('know', 314), ('us', 304), ('believe', 298), ('christ', 281), ('say', 280), ('also', 277),

Для класса comp.sys.mac.hardware наиболее частотными словами являются слова, связанные с компьютерной техникой, для двух других классов - слова, связанные с религией. Так как последних большинство, то они являются наиболее частотными для всей выборки.

Значение коэффициент Жаккара между всеми классами уменьшилось, связано это с тем, что стоп-слов, которые одинаковы для всех классов, теперь нет.

#### Со стеммингом и с отсечением стоп слов

In [81]:
print_frequency_words(stem_train, class_num=None, stop_words=mystopwords)
print_frequency_words(stem_train, class_num=0, stop_words=mystopwords)
print_frequency_words(stem_train, class_num=1, stop_words=mystopwords)
print_frequency_words(stem_train, class_num=2, stop_words=mystopwords)

print_jaccard_coef(stem_train, 0, 1, stop_words=mystopwords)
print_jaccard_coef(stem_train, 0, 2, stop_words=mystopwords)
print_jaccard_coef(stem_train, 1, 2, stop_words=mystopwords)

Первые 20 наиболее частотных слов всей выборки:
[('thi', 2494), ('wa', 1669), ('god', 1453), ('one', 1193), ('would', 1105), ('hi', 1004), ('christian', 909), ('ha', 867), ('doe', 788), ('peopl', 785), ('say', 759), ('use', 731), ('know', 720), ('jesu', 716), ('think', 659), ('ani', 658), ('onli', 625), ('like', 613), ('believ', 599), ('time', 532)]
Первые 20 наиболее частотных слов класса comp.sys.mac.hardware:
[('mac', 327), ('apple', 266), ('one', 246), ('drive', 211), ('would', 200), ('get', 178), ('use', 173), ('problem', 171), ('like', 163), ('know', 162), ('bit', 150), ('scsi', 142), ('system', 139), ('anyone', 123), ('thanks', 120), ('card', 115), ('32', 113), ('memory', 112), ('new', 110), ('also', 109)]
Первые 20 наиболее частотных слов класса soc.religion.christian:
[('god', 1097), ('would', 610), ('one', 608), ('jesus', 466), ('people', 452), ('church', 340), ('think', 337), ('know', 314), ('us', 304), ('believe', 298), ('christ', 281), ('say', 280), ('also', 277), ('time',

Здесь для всей выборки наиболее частотными словами являются усеченные слова. За счет этого также увеличилось значение коэффициента Жаррака между всеми классами.

### 6. Используя Pipeline, реализуем модель Наивного Байесовского классификатора и выявим на основе показателей качества (значения полноты, точности, f1-меры и аккуратности), какая предварительная обработка данных обеспечит наилучшие результаты классификации.

При проведении данного исследования фиксируем все переменные кроме одной, а далее меняем незафиксированную переменную для определения ее наилучего значения. После того как наилучшее значение найдено, фиксируем это значение, и переходим к следующей переменной.

In [84]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

def pipeline(x_train, y_train, x_test, stop_words=None, max_features=1000, use_tf=True, use_idf=True):
    if use_tf:
        text_clf = Pipeline([('vect', CountVectorizer(max_features=max_features, stop_words=stop_words)),
                             ('tfidf', TfidfTransformer(use_idf=use_idf)),
                             ('clf', MultinomialNB()),], 
                             verbose=10)  
    else:
        text_clf = Pipeline([('vect', CountVectorizer(max_features=max_features, stop_words=stop_words)),
                             ('clf', MultinomialNB()),],
                             verbose=10) 
        
    text_clf = text_clf.fit(x_train, y_train)
    prediction = text_clf.predict(x_test)
    return prediction

In [85]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def print_score(prediction, y_test):
    '''Расчет показателей качества'''
    precision = round(precision_score(prediction, y_test, average='weighted'), 3)
    print ('Precision score: ', precision)
    recall = round(recall_score(prediction, y_test, average='weighted'), 3)
    print ('Recall score: ', recall)
    f1 = round(f1_score(prediction, y_test, average='weighted'), 3)
    print ('F1-score: ', f1)
    accuracy = round(accuracy_score(prediction, y_test), 3)
    print ('Accuracy score: ', accuracy)

    return [precision, recall, f1, accuracy]

In [86]:
columns = ['precision', 'recall', 'f1-score', 'accuracy']
df = pd.DataFrame(columns=columns)

#### Наличие \ отсутствие стемминга

In [87]:
prediction = pipeline(twenty_train.data, twenty_train.target, twenty_test.data)
results_withot_stem = print_score(prediction, twenty_test.target)
df.loc['results_withot_stem'] = results_withot_stem

[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.0s
Precision score:  0.914
Recall score:  0.752
F1-score:  0.802
Accuracy score:  0.752


In [98]:
prediction = pipeline(stem_train, twenty_train.target, stem_test)
results_stem = print_score(prediction, twenty_test.target)
df.loc['results_stem'] = results_stem

[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.0s
Precision score:  0.917
Recall score:  0.766
F1-score:  0.811
Accuracy score:  0.766


#### Отсечение \ не отсечение стоп-слов

In [99]:
prediction = pipeline(stem_train, twenty_train.target, stem_test, stop_words=mystopwords)
results_stop_words = print_score(prediction, twenty_test.target)
df.loc['results_stop_words'] = results_stop_words

[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.0s
Precision score:  0.908
Recall score:  0.781
F1-score:  0.817
Accuracy score:  0.781


#### Взвешивание: Count, TF, TF-IDF

In [100]:
prediction = pipeline(stem_train, twenty_train.target, stem_test, stop_words=mystopwords, use_tf=True, use_idf=False)
results_tf = print_score(prediction, twenty_test.target)
df.loc['results_tf'] = results_stop_words

[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.0s
Precision score:  0.917
Recall score:  0.77
F1-score:  0.814
Accuracy score:  0.77


In [101]:
prediction = pipeline(stem_train, twenty_train.target, stem_test, stop_words=mystopwords, use_tf=False, use_idf=False)
results_count = print_score(prediction, twenty_test.target)
df.loc['results_count'] = results_stop_words

[Pipeline] .............. (step 1 of 2) Processing vect, total=   0.2s
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.0s
Precision score:  0.769
Recall score:  0.757
F1-score:  0.761
Accuracy score:  0.757


#### Количество информативных терминов (max_features) - исследовано 5 значений в диапазоне от 100 до общего количества слов в выборке.

In [102]:
max_featurs = [100, 1000, 2000, 5000, 10000]
for i in max_featurs:
    prediction = pipeline(stem_train, twenty_train.target, stem_test, stop_words=mystopwords, max_features=i)
    results = print_score(prediction, twenty_test.target)
    df.loc[f'results_{i}'] = results

[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.0s
Precision score:  0.87
Recall score:  0.676
F1-score:  0.75
Accuracy score:  0.676
[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.0s
Precision score:  0.908
Recall score:  0.781
F1-score:  0.817
Accuracy score:  0.781
[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.0s
Precision score:  0.914
Recall score:  0.781
F1-score:  0.818
Accuracy score:  0.781
[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.2s
[Pipeline] ............. (step 2 of 3

In [103]:
df

,precision,recall,f1-score,accuracy
results_withot_stem,0.914,0.752,0.802,0.752
results_stem,0.917,0.766,0.811,0.766
results_stop_words,0.908,0.781,0.817,0.781
results_tf,0.908,0.781,0.817,0.781
results_count,0.908,0.781,0.817,0.781
results_100,0.870,0.676,0.750,0.676
results_1000,0.908,0.781,0.817,0.781
results_2000,0.914,0.781,0.818,0.781
results_5000,0.945,0.757,0.817,0.757
results_10000,0.964,0.738,0.817,0.738


**Вывод:** 
- Отсечение стоп-слов позволяет избавится от неинформативных признаков;
- Стемминг служит для повышения информативности признаков, так как группируются близкие по значению слова, исходя из их общей основы;
- Count - самый простой способ векторизации, который заключается в подсчете терминов в документе, TF - частота появления термина в конкретном документе, TF-IDF - позволяет учесть важность термина в документе, а не только частоту его появления;
- max_features позволяет ограничить количество информативных признаков

Таким образом, наиболее подходящая комбинация параметров следующая: Наличие стемминга, с отсечением стоп слов, TF-IDF векторизация, количество информативных признаков 2000